In [385]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [386]:
import os
os.chdir("/content/drive/MyDrive")

In [387]:
#Importing libraries
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
import pickle
from sklearn.linear_model import Lasso, Ridge, LogisticRegression,SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

In [388]:
#Load the datasets test csv and train csv and display the first few rows to examine the reviews
df_train = pd.read_csv("train_a.csv")
df_test = pd.read_csv("test_a.csv")

In [389]:
#Display first few rows of train data using head()
df_train.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [390]:
#Display first few rows of test data using head()
df_test.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,3,5,5,5,5,2,5,5,50,44.0
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,5,4,4,4,4,3,4,5,0,0.0
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,2,4,1,3,2,2,2,0,0.0
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,4,1,1,1,1,3,1,4,0,6.0
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,2,4,2,4,0,20.0


In [391]:
df_train.shape

(103904, 25)

In [392]:
df_test.shape

(25976, 24)

In [393]:
df_train.dtypes

Unnamed: 0                             int64
id                                     int64
Gender                                object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Inflight wifi service                  int64
Departure/Arrival time convenient      int64
Ease of Online booking                 int64
Gate location                          int64
Food and drink                         int64
Online boarding                        int64
Seat comfort                           int64
Inflight entertainment                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Inflight service                       int64
Cleanliness                            int64
Departure 

In [394]:
df_test.dtypes

Unnamed: 0                             int64
id                                     int64
Gender                                object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Inflight wifi service                  int64
Departure/Arrival time convenient      int64
Ease of Online booking                 int64
Gate location                          int64
Food and drink                         int64
Online boarding                        int64
Seat comfort                           int64
Inflight entertainment                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Inflight service                       int64
Cleanliness                            int64
Departure 

In [395]:
# Checking for null values in train data
df_train.isna().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction    

In [396]:
# Checking for null values in test data
df_test.isna().sum()

Unnamed: 0                            0
id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
dtype: int64

In [397]:
# drop = Unnamed: 0,id,Gender, Arrival Delay in Minutes, Departure Delay in Minutes, Checkin service, Baggage handling, Leg room service, On-board service, Inflight entertainment, Online boarding, Gate location, Ease of Online booking, Departure/Arrival time convenient
# cont =  Flight Distance,  Age
# ordinal enc = Inflight service
# onehot enc = Class, Type of Travel, Customer Type

In [398]:
#Dropping irrelevent columns from train data
df_train.drop(["Unnamed: 0","id","Departure/Arrival time convenient","Gate location","Online boarding","Inflight wifi service","Cleanliness","Food and drink","Ease of Online booking","Seat comfort","Inflight entertainment","On-board service","Leg room service","Baggage handling","Checkin service","Departure Delay in Minutes","Arrival Delay in Minutes"],axis = 1,inplace=True)
df_train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight service,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,5,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,4,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,4,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,4,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,satisfied


In [399]:
#Dropping irrelevent columns from test data
df_test.drop(["Unnamed: 0","id","Departure/Arrival time convenient","Gate location","Online boarding","Inflight wifi service","Cleanliness","Food and drink","Ease of Online booking","Seat comfort","Inflight entertainment","On-board service","Leg room service","Baggage handling","Checkin service","Departure Delay in Minutes","Arrival Delay in Minutes"],axis = 1,inplace=True)
df_test.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight service
0,Female,Loyal Customer,52,Business travel,Eco,160,5
1,Female,Loyal Customer,36,Business travel,Business,2863,4
2,Male,disloyal Customer,20,Business travel,Eco,192,2
3,Male,Loyal Customer,44,Business travel,Business,3377,1
4,Female,Loyal Customer,49,Business travel,Eco,1182,2


In [400]:
#Preprocessing for train data

#one hot encoding for train data

#Encode categorical variables for train data

#Apply one hot encoding
one_hot_encoder = OneHotEncoder(drop = "first",sparse=False, dtype=int)

#Fit and transform the "Gender","Customer Type","Type of Travel","Class","Inflight wifi service","Ease of Online booking","Food and drink","Cleanliness" column needed for training data
col_enc_df = one_hot_encoder.fit_transform(df_train[["Customer Type","Type of Travel","Class","Gender"]])

# Get the column names for the one-hot encoded variables
churn_columns = one_hot_encoder.get_feature_names_out(["Customer Type","Type of Travel","Class","Gender"])

# Create a DataFrame with the encoded data
col_enc_df1 = pd.DataFrame(col_enc_df, columns=churn_columns)

# Drop the original "Geography","Gender","NumOfProducts" column and concatenate the new encoded columns
df_encoded = pd.concat([df_train.drop(columns=["Customer Type","Type of Travel","Class","Gender"]), col_enc_df1], axis=1)

# Display the encoded DataFrame
df_encoded.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,13,460,5,neutral or dissatisfied,0,1,0,1,1
1,25,235,4,neutral or dissatisfied,1,0,0,0,1
2,26,1142,4,satisfied,0,0,0,0,0
3,25,562,4,neutral or dissatisfied,0,0,0,0,0
4,61,214,3,satisfied,0,0,0,0,1


In [401]:
#Ordinal encoding for train data

# Select the column to encode
inflight_service_col = df_encoded[['Inflight service']]

# Initialize the OrdinalEncoder
encoder = OrdinalEncoder()

# Fit and transform the column
encoded_inflight_service = encoder.fit_transform(inflight_service_col)

# Replace the original column with the encoded values
df_encoded['Inflight service'] = encoded_inflight_service

# Display the first few rows to confirm the changes
df_encoded.head()

,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,13,460,5.0,neutral or dissatisfied,0,1,0,1,1
1,25,235,4.0,neutral or dissatisfied,1,0,0,0,1
2,26,1142,4.0,satisfied,0,0,0,0,0
3,25,562,4.0,neutral or dissatisfied,0,0,0,0,0
4,61,214,3.0,satisfied,0,0,0,0,1


In [402]:
# Scaling for train data

#Identify columns to scale
continuous_features = ["Flight Distance","Age"]

#Apply scaling only to the numerical features

#Fit and Transform needed for training data
scaler = StandardScaler()
df_encoded[continuous_features] = scaler.fit_transform(df_encoded[continuous_features])
df_encoded.head()

,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,-1.745279,-0.731539,5.0,neutral or dissatisfied,0,1,0,1,1
1,-0.951360,-0.957184,4.0,neutral or dissatisfied,1,0,0,0,1
2,-0.885200,-0.047584,4.0,satisfied,0,0,0,0,0
3,-0.951360,-0.629246,4.0,neutral or dissatisfied,0,0,0,0,0
4,1.430397,-0.978244,3.0,satisfied,0,0,0,0,1


In [403]:
#Encoding test data

# Transform the "Geography","Gender","NumOfProducts" column for test data
encoded_new = one_hot_encoder.transform(df_test[["Customer Type","Type of Travel","Class","Gender"]])

# Get the column names for the one-hot encoded variables
columns_new = one_hot_encoder.get_feature_names_out(["Customer Type","Type of Travel","Class","Gender"])

# Create a DataFrame with the encoded data
encoded_df_new = pd.DataFrame(encoded_new, columns=columns_new)

# Drop the original "Geography","Gender","NumOfProducts" column and concatenate the new encoded columns
df_encoded_new = pd.concat([df_test.drop(columns=["Customer Type","Type of Travel","Class","Gender"]), encoded_df_new], axis=1)
df_encoded_new.head()

,Age,Flight Distance,Inflight service,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,52,160,5,0,0,1,0,0
1,36,2863,4,0,0,0,0,0
2,20,192,2,1,0,1,0,1
3,44,3377,1,0,0,0,0,1
4,49,1182,2,0,0,1,0,0


In [404]:
#Ordinal encoding for test data

# Select the column to encode
inflight_service_col_new = df_encoded_new[['Inflight service']]

# # Initialize the OrdinalEncoder
# encoder = OrdinalEncoder()

# Fit and transform the column
encoded_inflight_service_new = encoder.transform(inflight_service_col_new)

# Replace the original column with the encoded values
df_encoded_new['Inflight service'] = encoded_inflight_service_new

# Display the first few rows to confirm the changes
df_encoded_new.head()

,Age,Flight Distance,Inflight service,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,52,160,5.0,0,0,1,0,0
1,36,2863,4.0,0,0,0,0,0
2,20,192,2.0,1,0,1,0,1
3,44,3377,1.0,0,0,0,0,1
4,49,1182,2.0,0,0,1,0,0


In [405]:
#Scaling test data

df_encoded_new[continuous_features] = scaler.transform(df_encoded_new[continuous_features])
df_encoded_new.head()

,Age,Flight Distance,Inflight service,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,0.834958,-1.032398,5.0,0,0,1,0,0
1,-0.223601,1.678348,4.0,0,0,0,0,0
2,-1.282160,-1.000307,2.0,1,0,1,0,1
3,0.305678,2.193821,1.0,0,0,0,0,1
4,0.636478,-0.007470,2.0,0,0,1,0,0


In [406]:
# Separate features and traget variable for train data
X = df_encoded.drop(["satisfaction"],axis = 1)
y = df_encoded["satisfaction"]

In [407]:
#Features for test data
X_test = df_encoded_new

In [408]:
X

,Age,Flight Distance,Inflight service,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,-1.745279,-0.731539,5.0,0,1,0,1,1
1,-0.951360,-0.957184,4.0,1,0,0,0,1
2,-0.885200,-0.047584,4.0,0,0,0,0,0
3,-0.951360,-0.629246,4.0,0,0,0,0,0
4,1.430397,-0.978244,3.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
103899,-1.083680,-1.000307,3.0,1,0,1,0,0
103900,0.636478,1.160869,5.0,0,0,0,0,1
103901,-0.620561,0.807860,5.0,1,0,0,0,1
103902,-1.149840,-0.189991,4.0,1,0,1,0,0


In [409]:
X_test

,Age,Flight Distance,Inflight service,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,0.834958,-1.032398,5.0,0,0,1,0,0
1,-0.223601,1.678348,4.0,0,0,0,0,0
2,-1.282160,-1.000307,2.0,1,0,1,0,1
3,0.305678,2.193821,1.0,0,0,0,0,1
4,0.636478,-0.007470,2.0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
25971,-0.355921,-0.665350,5.0,1,0,0,0,1
25972,-1.083680,-0.545006,5.0,0,0,0,0,1
25973,-1.480640,-0.362484,4.0,0,1,1,0,0
25974,-1.679119,-0.062627,5.0,0,0,0,0,1


In [410]:
#Decision Tree

#Train a Decision Tree using the data set.

#Define parameter grid for Decision Tree
param_grid_dt = {
    'criterion': ['gini','entropy'],
    'max_depth': [10, 20, 30]
}
#Initialize GridSearchCV for Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)

#GridSearchCV helps in finding the optimal set of hyperparameters that yield the best performance.
#It exhaustively searches over a specified parameter grid.Instead of manually trying different combinations, GridSearchCV automates this process, ensuring that all combinations are evaluated.

#Initialize GridSearchCV
grid_search_dt = GridSearchCV(dt_model, param_grid_dt, cv=5, scoring='accuracy')

#Fit GridSearchCV
grid_search_dt.fit(X, y)

grid_search_dt.cv_results_

#Best Decision Tree model
best_dt_model = grid_search_dt.best_estimator_

#Once Decision Tree model is trained using GridSearchCV and obtained the best model (best_dt_model), we can use it to make predictions on new data.

#Predicting on the test set
y_test_pred = best_dt_model.predict(X_test)
X_test['satisfaction_dt']=  y_test_pred

#Save pred as csv
# X_test.to_csv('/content/drive/MyDrive/satisfaction_Predictions_1.csv', index=False)

#Predicting on the train set
y_train_pred = best_dt_model.predict(X)
#Report the model accuracy, confusion matrix, and classification report for each model.
#Evaluation
print("Decision Tree Performance:")
print("Accuracy:", accuracy_score(y, y_train_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_train_pred))
print("Classification Report:\n", classification_report(y, y_train_pred))

Decision Tree Performance:
Accuracy: 0.8481194179242377
Confusion Matrix:
 [[54087  4792]
 [10989 34036]]
Classification Report:
                          precision    recall  f1-score   support

neutral or dissatisfied       0.83      0.92      0.87     58879
              satisfied       0.88      0.76      0.81     45025

               accuracy                           0.85    103904
              macro avg       0.85      0.84      0.84    103904
           weighted avg       0.85      0.85      0.85    103904



In [411]:
import pandas as pd
import matplotlib.pyplot as plt
importances = best_dt_model.feature_importances_

feature_imp_df = pd.DataFrame({'Feature': X.columns, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False)
print(feature_imp_df)

                           Feature  Gini Importance
4   Type of Travel_Personal Travel         0.340979
2                 Inflight service         0.255891
3  Customer Type_disloyal Customer         0.183467
0                              Age         0.103462
5                        Class_Eco         0.060963
6                   Class_Eco Plus         0.025773
1                  Flight Distance         0.023245
7                      Gender_Male         0.006220


In [412]:
#Logistic Regression

# Ensure correct parameter grid
log_reg_params = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear']
    }

# Logistic Regression with GridSearchCV (The GridSearchCV is configured to try these combinations with 5-fold cross-validation)
# Error Handling parameter is set to 'raise' to immediately raise an error if a fit fails, which can help in debugging.
log_reg = LogisticRegression()
log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, error_score='raise')
log_reg_grid.fit(X, y)

print(log_reg_grid.cv_results_)

best_log_reg = log_reg_grid.best_estimator_

print(best_log_reg.predict_proba(X)[:,1]/best_log_reg.predict_proba(X)[:,0])

#Best parameters and scores
print("Best Logistic Regression Parameters:", log_reg_grid.best_params_)
print("Best Logistic Regression Score:", log_reg_grid.best_score_)

#Predict on the test set
y_test_pred_log = best_log_reg.predict(X_test.drop(columns=['satisfaction_dt']))
# y_test_pred_log = best_log_reg.predict(X_test)
X_test['satisfaction_log']=  y_test_pred_log

#Save the predictions to CSV files
X_test.to_csv('predictions_airlines.csv', index=False)

#Predict on the train set
y_train_pred_new = best_log_reg.predict(X)

#Evaluation
accuracy_log_reg = accuracy_score(y, y_train_pred_new)
conf_matrix_log_reg = confusion_matrix(y, y_train_pred_new)
class_report_log_reg = classification_report(y, y_train_pred_new)

print("Logistic Regression Accuracy:", accuracy_log_reg)
print("Logistic Regression Confusion Matrix:\n", conf_matrix_log_reg)
print("Logistic Regression Classification Report:\n", class_report_log_reg)

{'mean_fit_time': array([0.38950543, 0.61169715, 0.28100429, 0.49495268, 0.27820082,
       0.51317739, 0.43563461, 0.56473041]), 'std_fit_time': array([0.04767997, 0.09573171, 0.00326277, 0.03379874, 0.00249366,
       0.03595923, 0.01187114, 0.10687567]), 'mean_score_time': array([0.12253232, 0.10809598, 0.08601069, 0.0863183 , 0.08536839,
       0.09565554, 0.11844726, 0.08851695]), 'std_score_time': array([0.02143418, 0.02447863, 0.00257547, 0.00354075, 0.00226033,
       0.0213352 , 0.02405051, 0.00192373]), 'param_C': masked_array(data=[0.01, 0.01, 0.1, 0.1, 1, 1, 10, 10],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_penalty': masked_array(data=['l2', 'l1', 'l2', 'l1', 'l2', 'l1', 'l2', 'l1'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_solver': masked_array(data=['liblinear', 'liblinear', 'liblinear', 'li

In [413]:
best_log_reg.coef_

array([[ 0.04227148,  0.03058816,  0.57003095, -1.87601093, -2.47723705,
        -1.00672681, -1.04169771, -0.00612006]])

In [414]:
best_log_reg.feature_names_in_

array(['Age', 'Flight Distance', 'Inflight service',
       'Customer Type_disloyal Customer',
       'Type of Travel_Personal Travel', 'Class_Eco', 'Class_Eco Plus',
       'Gender_Male'], dtype=object)

In [415]:
new_pass = {'Gender': 'Female',
  'Customer Type': 'Loyal Customer',
  'Age': 31,
  'Type of Travel': 'Personal Travel',
  'Class': 'Eco',
  'Flight Distance': 259,
  'Inflight wifi service': 3,
  'Departure/Arrival time convenient': 3,
  'Ease of Online booking': 3,
  'Gate location': 2,
  'Food and drink': 1,
  'Online boarding': 3,
  'Seat comfort': 1,
  'Inflight entertainment': 1,
  'On-board service': 1,
  'Leg room service': 3,
  'Baggage handling': 4,
  'Checkin service': 2,
  'Inflight service': 2,
  'Cleanliness': 1,
  'Departure Delay in Minutes': 3,
  'Arrival Delay in Minutes': 4.0,
  'satisfaction': 'neutral or dissatisfied'}

df_new = pd.DataFrame([new_pass])
df_new.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Female,Loyal Customer,31,Personal Travel,Eco,259,3,3,3,2,...,1,1,3,4,2,2,1,3,4.0,neutral or dissatisfied


In [416]:
df_new.isna().sum()
df_new.shape
df_new.dtypes

Gender                                object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Inflight wifi service                  int64
Departure/Arrival time convenient      int64
Ease of Online booking                 int64
Gate location                          int64
Food and drink                         int64
Online boarding                        int64
Seat comfort                           int64
Inflight entertainment                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Inflight service                       int64
Cleanliness                            int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
satisfacti

In [417]:
#Dropping irrelevent columns
df_new.drop(["Departure/Arrival time convenient","Gate location","Online boarding","Inflight wifi service","Cleanliness","Food and drink","Ease of Online booking","Seat comfort","Inflight entertainment","On-board service","Leg room service","Baggage handling","Checkin service","Departure Delay in Minutes","Arrival Delay in Minutes"],axis = 1,inplace=True)
df_new.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight service,satisfaction
0,Female,Loyal Customer,31,Personal Travel,Eco,259,2,neutral or dissatisfied


In [418]:
#Encoding test data

# Transform the "Geography","Gender","NumOfProducts" column for test data
encoded_new_data = one_hot_encoder.transform(df_new[["Customer Type","Type of Travel","Class","Gender"]])

# Get the column names for the one-hot encoded variables
columns_new_data = one_hot_encoder.get_feature_names_out(["Customer Type","Type of Travel","Class","Gender"])

# Create a DataFrame with the encoded data
encoded_df_new_data = pd.DataFrame(encoded_new_data, columns=columns_new_data)

# Drop the original "Geography","Gender","NumOfProducts" column and concatenate the new encoded columns
df_encoded_new_data = pd.concat([df_new.drop(columns=["Customer Type","Type of Travel","Class","Gender"]), encoded_df_new_data], axis=1)
df_encoded_new_data.head()

,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,31,259,2,neutral or dissatisfied,0,1,1,0,0


In [419]:
#Ordinal encoding for test data

# Select the column to encode
inflight_service_col_new_data = df_encoded_new_data[['Inflight service']]

# # Initialize the OrdinalEncoder
# encoder = OrdinalEncoder()

# Fit and transform the column
encoded_inflight_service_new_test = encoder.transform(inflight_service_col_new_data)

# Replace the original column with the encoded values
df_encoded_new_data['Inflight service'] = encoded_inflight_service_new_test

# Display the first few rows to confirm the changes
df_encoded_new_data.head()

,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,31,259,2.0,neutral or dissatisfied,0,1,1,0,0


In [420]:
#Scaling test data

df_encoded_new_data[continuous_features] = scaler.transform(df_encoded_new_data[continuous_features])
df_encoded_new_data.head()

,Age,Flight Distance,Inflight service,satisfaction,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,Gender_Male
0,-0.554401,-0.933115,2.0,neutral or dissatisfied,0,1,1,0,0


In [421]:
# Separate features and traget variable for new test data
X_new = df_encoded_new_data.drop(["satisfaction"],axis = 1)
y_new = df_encoded_new_data["satisfaction"]

In [422]:
# Predict new data point using Decision Tree

y_test_pred_new = best_dt_model.predict(X_new)
X_new['satisfaction_dt']=  y_test_pred_new

# prediction = best_model.predict(df_new)[0]

print(f"Prediction: {'Satisfied' if y_test_pred_new == 1 else 'Not Satisfied'}")

# Recommendation
if y_test_pred_new == 1:
    print("Recommendation: Continue to monitor customer satisfaction and engagement levels.")
else:
    print("Recommendation: Engage with the customer through personalized offers and improved service to retain them.")


Prediction: Not Satisfied
Recommendation: Engage with the customer through personalized offers and improved service to retain them.


In [423]:
# Predict new data point using Log Reg

#Predict on the test set
y_test_pred_log_new = best_log_reg.predict(X_new.drop(columns=['satisfaction_dt']))
# y_test_pred_log = best_log_reg.predict(X_test)
X_new['satisfaction_log']=  y_test_pred_log_new

#Save the predictions to CSV files
X_new.to_csv('predictions_new_data.csv', index=False)

print(f"Prediction: {'Satisfied' if y_test_pred_log_new == 1 else 'Not Satisfied'}")

# Recommendation
if y_test_pred_log_new == 1:
    print("Recommendation: Continue to monitor customer satisfaction and engagement levels.")
else:
    print("Recommendation: Engage with the customer through personalized offers and improved service to retain them.")


Prediction: Not Satisfied
Recommendation: Engage with the customer through personalized offers and improved service to retain them.


In [ ]:
# LogisticRegression

In [ ]:
# 1)Identifying Key Factors: Use statistical analysis or machine learning techniques to determine the significant drivers of passenger satisfaction within our dataset.
# Summarize your findings, highlighting the primary factors with quantitative evidence from the analysis.

# To determine the significant drivers of passenger satisfaction within the dataset, we can use statistical analysis and machine learning techniques.

# From the logistic regression output, we have the following results:

# Coefficients: array([[ 0.04227148, 0.03058816, 0.57003095, -1.87601093, -2.47723705, -1.00672681, -1.04169771, -0.00612006]])
# Features: array(['Age', 'Flight Distance', 'Inflight service', 'Customer Type_disloyal Customer', 'Type of Travel_Personal Travel', 'Class_Eco', 'Class_Eco Plus', 'Gender_Male'], dtype=object)

# Interpretation of Coefficients
# The coefficients indicate the change in the log odds of the target variable (satisfaction) for a one-unit increase in the corresponding predictor variable, holding all other variables constant.

# Significant Drivers of Passenger Satisfaction

# Inflight Service:

# Coefficient: 0.570
# Interpretation: A higher inflight service rating significantly increases the likelihood of a passenger being satisfied. This is the strongest positive predictor among the variables considered.

# Customer Type (Disloyal Customer):

# Coefficient: -1.876
# Interpretation: Being a disloyal customer significantly decreases the likelihood of satisfaction. This is a strong negative predictor.

# Gender (Male):

# Coefficient: -0.006
# Interpretation: Gender has a very small and negligible effect on satisfaction.

# Flight Distance:

# Coefficient: 0.031
# Interpretation: Longer flight distances slightly increase the likelihood of satisfaction.

# Type of Travel (Personal Travel):

# Coefficient: -2.477
# Interpretation: Passengers traveling for personal reasons are significantly less likely to be satisfied compared to business travelers. This is the strongest negative predictor.

# Based on the logistic regression analysis:

# Inflight Service is the most significant positive driver of passenger satisfaction.
# Customer Type (Disloyal) and Type of Travel (Personal) are the most significant negative drivers of satisfaction.
# Travel Class also plays a crucial role, with Economy and Economy Plus classes being associated with lower satisfaction compared to Business class.
# Age and Flight Distance have positive but smaller effects on satisfaction.
# Gender has a negligible impact on satisfaction.

# Quantitative Evidence from Logistic Regression

# Model Accuracy: 79.32%

# Logistic Regression Confusion Matrix:
#  [[47430 11449]
#  [10036 34989]]
# Logistic Regression Classification Report:
#                           precision    recall  f1-score   support

# neutral or dissatisfied       0.83      0.81      0.82     58879
#               satisfied       0.75      0.78      0.77     45025

#                accuracy                           0.79    103904
#               macro avg       0.79      0.79      0.79    103904
#            weighted avg       0.79      0.79      0.79    103904


In [ ]:
# 2) Reasoning Behind Satisfaction: Interpret your analysis and explain the business or logical reasoning behind why the identified factors are critical to passenger satisfaction. Structure your response to
# connect each factor with specific aspects of the airline customer experience.

# The logistic regression analysis identified several key factors influencing passenger satisfaction.

# Inflight Service
# Business Reasoning: Inflight service encompasses various elements such as the quality of food and beverages, attentiveness and friendliness of the cabin crew, and availability of amenities like Wi-Fi and entertainment options.
# Impact on Customer Experience: Passengers who receive high-quality inflight service are more likely to feel valued and comfortable, leading to higher satisfaction. Exceptional service can differentiate an airline from its competitors and foster customer loyalty.

# Customer Type (Disloyal Customer)
# Business Reasoning: Loyal customers typically receive benefits like priority boarding, additional baggage allowance, and access to lounges, which enhance their overall travel experience.
# Impact on Customer Experience: Disloyal customers, who do not receive these perks, may feel less valued and less likely to have their expectations met. As a result, they are more likely to report lower satisfaction levels. Focusing on converting disloyal customers to loyal ones through reward programs and personalized service can improve satisfaction rates.

# Type of Travel (Personal Travel)
# Business Reasoning: Business travelers often have their trips organized and paid for by their companies, with travel arrangements that typically include business class seats and premium services. Personal travelers, on the other hand, may be more cost-conscious and opt for economy class.
# Impact on Customer Experience: Personal travelers may have higher expectations for comfort and service relative to what they are willing to pay, leading to a mismatch between expectations and actual experience. Airlines can address this by offering targeted promotions and affordable upgrades to enhance the travel experience for personal travelers.

# Travel Class (Economy and Economy Plus)
# Business Reasoning: Travel class significantly affects the level of comfort and service a passenger receives. Business class offers more legroom, better seating, superior meals, and exclusive services, while economy and economy plus provide more basic amenities.
# Impact on Customer Experience: Passengers in economy and economy plus classes are likely to experience less comfort and fewer amenities, which can lead to lower satisfaction. Enhancing the economy class experience through small upgrades like improved seating, better meal options, and attentive service can help improve satisfaction levels.

# Age
# Business Reasoning: Older passengers may have different expectations and requirements compared to younger passengers, such as a preference for more comfortable seating, assistance with boarding, and special meal options.
# Impact on Customer Experience: Airlines that cater to the needs of older passengers by providing comfortable seating, assistance services, and tailored amenities can enhance satisfaction among this demographic. Understanding and addressing age-specific needs can lead to higher overall satisfaction.

# Flight Distance
# Business Reasoning: Longer flights require better planning and provision of amenities to keep passengers comfortable over extended periods. Passengers on longer flights may place more importance on inflight entertainment, meal quality, and seat comfort.
# Impact on Customer Experience: Ensuring that long-haul flights offer high-quality services, such as comfortable seating, diverse entertainment options, and frequent meal and beverage services, can significantly enhance passenger satisfaction. Airlines should focus on maintaining and improving the inflight experience for longer routes to meet passenger expectations.

# Gender
# Business Reasoning: While gender was found to have a negligible impact on satisfaction, it is still important to consider the diverse needs and preferences of all passengers.
# Impact on Customer Experience: Offering a balanced and inclusive range of services and amenities that cater to all passengers can contribute to a more universally satisfying experience. Airlines should ensure that their service quality is consistently high and inclusive, regardless of passenger demographics.

In [ ]:
# 3)Insights and Recommendations: Based on your analysis, propose actionable recommendations for the airline to improve customer satisfaction. Describe how each recommendation could be implemented,
# emphasizing practicality, feasibility, and potential impact such as its associated costs and benefits.

# Based on the analysis, we propose several actionable recommendations for the airline to improve customer satisfaction. Each recommendation is designed to be practical, feasible, and to provide a significant impact on customer satisfaction.

# Enhance Inflight Service Quality
# Insight: Inflight service quality is the most significant positive driver of passenger satisfaction.

# Recommendation:

# Enhance Inflight Service Quality: Improve food, beverage options, train cabin crew, and upgrade amenities.
# Develop Loyalty Programs: Introduce frequent flyer programs and offer personalized services.
# Enhance Economy Class Experience: Improve seating comfort and offer small upgrades.
# Cater to Personal Travelers: Create travel packages and provide travel perks.
# Tailor Services for Older Passengers: Offer assistance services and develop age-specific amenities.
# Improve Long-Haul Flight Experience: Enhance comfort, entertainment, and provide wellness kits.

In [ ]:
# Decision Tree:

In [ ]:
# 1)Identifying Key Factors: Use statistical analysis or machine learning techniques to determine the significant drivers of passenger satisfaction within our dataset. Summarize your findings, highlighting the primary factors with quantitative evidence from the analysis.

# Summary of Findings:
# Type of Travel (Personal vs. Business) emerges as the most critical factor influencing passenger satisfaction. This suggests that personal travelers may have different expectations or experiences compared to business travelers.

# Inflight service ranks closely behind, implying that the quality and delivery of inflight services significantly impact satisfaction levels.

# Customer Type (particularly disloyal customers) also plays a notable role, indicating that managing and understanding customer loyalty dynamics is crucial for enhancing satisfaction.

# Demographic factors such as Age and Gender have varying impacts, with age showing a more substantial influence than gender in this analysis.

In [ ]:
# 2)Reasoning Behind Satisfaction: Interpret your analysis and explain the business or logical reasoning behind why the identified factors are critical to passenger satisfaction.
# Structure your response to connect each factor with specific aspects of the airline customer experience.

# Type of Travel (Personal vs. Business):
# Reasoning: Personal and business travelers have distinct expectations and needs. Personal travelers might prioritize comfort, relaxation, and seamless travel experiences, whereas business travelers often prioritize efficiency, timely service, and amenities that support productivity.

# Inflight service:
# Reasoning: Inflight services encompass everything from meal quality to onboard entertainment and the attentiveness of cabin crew. These elements directly impact passenger comfort and satisfaction during the flight.

# Customer Type (disloyal Customer):
# Reasoning: Disloyal customers may have had previous negative experiences or perceive better value elsewhere. Understanding and addressing their concerns can prevent churn and potentially convert them into loyal customers.

# Age:
# Reasoning: Different age groups may have varying expectations and preferences when it comes to travel experiences. Younger passengers might value digital connectivity and entertainment options, while older passengers might prioritize comfort and assistance.

# Class (Economy and Economy Plus):
# Reasoning: Class distinctions affect the level of comfort, space, and service quality experienced by passengers. Economy and Economy Plus passengers have different expectations regarding seat comfort, legroom, meal options, and overall service.

# Flight Distance:
# Reasoning: Longer flights generally require more extensive service offerings and amenities to keep passengers comfortable and satisfied throughout the journey.

# Gender:
# Reasoning: While gender differences in travel preferences exist, they may not be as pronounced as other factors in this analysis. Factors like seating preferences, onboard service interactions, and amenity usage might vary slightly based on gender but are generally overshadowed by other more impactful variables.


In [ ]:
# 3)Insights and Recommendations: Based on your analysis, propose actionable recommendations for the airline to improve customer satisfaction. Describe how each recommendation could be implemented, emphasizing practicality, feasibility, and potential impact such as its associated costs and benefits.

# Insights from the analysis highlight several critical factors influencing passenger satisfaction in airlines, including the type of travel (personal vs. business), inflight services, customer loyalty dynamics, age demographics, class distinctions, and to a lesser extent, gender. These factors underscore the diverse expectations and preferences of passengers, necessitating tailored strategies to enhance satisfaction and loyalty.

# Based on these insights, actionable recommendations for airlines include:

# Enhancing Personal Traveler Experience: Tailor services for personal travelers, emphasizing comfort and personalized service.

# Improving Inflight Services: Invest in meal quality, entertainment options, and cabin crew training across all classes.

# Targeting Disloyal Customers: Implement personalized retention strategies to regain trust and improve satisfaction among disloyal segments.

# Customizing Services by Age: Adapt amenities and services to meet the distinct expectations of different age groups.

# Differentiating Economy and Economy Plus: Enhance offerings and service levels in each class to better meet passenger expectations.

# Continuous Monitoring and Adaptation: Implement feedback mechanisms to monitor satisfaction levels and make proactive adjustments.

In [424]:
# 4)Case Analysis and Strategy Recommendations: Analyze the provided passenger profile, integrating insights from the previous questions to understand and address the reasons for dissatisfaction.
# Propose specific, actionable strategies to enhance this passenger’s future satisfaction, discussing the feasibility and expected benefits.

# Logistic regression provides clear coefficients for each feature, indicating the direction and strength of their impact on the outcome.
# For example, a positive coefficient means the feature increases the likelihood of exiting, and a negative coefficient means it decreases the likelihood.

# Decision trees are inherently different from linear models like logistic regression, and interpreting their "coefficients" directly isn't straightforward.
# Feature importance scores provide a way to understand which features contribute most to the predictions in a decision tree.

# While decision trees provide insights into feature importance and decision paths, logistic regression offers clear, interpretable coefficients that
# quantify the impact of each feature on the likelihood of exiting. The linearity and straightforward interpretation of logistic regression coefficients
# make it easier to understand and communicate the influence of each feature.

# Both the Decision Tree and Logistic regression models has predicted that the customer will not be satisfied. This means that based on the features provided (such as Inflight service	, Customer Type,
# Type of Travel, etc.), the model has determined that the customer is not likely to be satisfied with the services.

# Recommendation: Engage with the customer through personalized offers and improved service to retain them.

# Since the prediction indicates that the customer is not satisfied, the recommendation focuses on engaging and enhancing customers through personalized offers and improved service to retain them.